In [5]:
from dotenv import load_dotenv
load_dotenv()

False

In [6]:
import datasets
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.retrievers import BM25Retriever

knowledge_base = datasets.load_dataset("m-ric/huggingface_doc", split="train")
knowledge_base = knowledge_base.filter(lambda row: row["source"].startswith("huggingface/transformers"))
source_docs = [
    Document(page_content=doc["text"], metadata={"source": doc["source"].split("/")[1]})
    for doc in knowledge_base
]

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    add_start_index=True,
    strip_whitespace=True,
    separators=["\n\n", "\n", ".", " ", ""],
)
docs_processed = text_splitter.split_documents(source_docs)

In [7]:
from smolagents import Tool

class RetrieverTool(Tool):
    name = "retriever"
    description = "Uses semantic search to retrieve the parts of transformers documentation that could be most relevant to answer your query."
    inputs = {
        "query": {
            "type": "string",
            "description": "The query to perform. This should be semantically close to your target documents. Use the affirmative form rather than a question.",
        }
    }
    output_type = "string"

    def __init__(self, docs, **kwargs):
        super().__init__(**kwargs)
        self.retriever = BM25Retriever.from_documents(
            docs, k=10
        )

    def forward(self, query: str) -> str:
        assert isinstance(query, str), "Your search query must be a string"

        docs = self.retriever.invoke(
            query,
        )
        return "\nRetrieved documents:\n" + "".join(
            [
                f"\n\n===== Document {str(i)} =====\n" + doc.page_content
                for i, doc in enumerate(docs)
            ]
        )

retriever_tool = RetrieverTool(docs_processed)


In [8]:

from smolagents import HfApiModel, CodeAgent, LiteLLMModel
agent = CodeAgent(
    model=LiteLLMModel("anthropic/claude-3-5-sonnet-latest"),
    tools=[retriever_tool],
    max_iterations=4,
    verbose=True
)

agent_output = agent.run("For a transformers model training, which is slower, the forward or the backward pass?")
print(agent_output)


╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ For a transformers model training, which is slower, the forward or the backward pass?                           │
│                                                                                                                 │
╰─ LiteLLMModel - anthropic/claude-3-5-sonnet-latest ─────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 0 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
Let me search through the transformers documentation to find information about forward and backward passes in model
training.                                                                                                          
                                                                                                                   
Thought: I will use the retriever tool to search for information about forward and backward passes in transformers 
training.                                                                                                          
                                                                                                                   
Code:                                                                                                              
```py                                                                                                              
info = retriever(query="forward pass backward pass training speed performance")                                    
print(info)                                                                                                        
```<end_code>                                                                                                      
                                                                                                                   

╭─ Executing this code: ──────────────────────────────────────────────────────────────────────────────────────────╮
│   1 info = retriever(query="forward pass backward pass training speed performance")                             │
│   2 print(info)                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Execution logs:

Retrieved documents:


===== Document 0 =====
Saving all activations from the forward pass in order to compute the gradients during the backward pass can result 
in 
significant memory overhead. The alternative approach of discarding the activations and recalculating them when 
needed 
during the backward pass, would introduce a considerable computational overhead and slow down the training process.

===== Document 1 =====
- A train step function which combines the loss function and optimizer update, does the forward and backward pass 
and returns the updated parameters.

===== Document 2 =====
model_inputs = tokenizer(src_text, text_target=tgt_text, return_tensors="pt")

model(**model_inputs)  # forward pass
```

- Generation

===== Document 3 =====
## How to benchmark 🤗 Transformers models

The classes [`PyTorchBenchmark`] and [`TensorFlowBenchmark`] allow to flexibly benchmark 🤗 Transformers models. 
The benchmark classes allow us to measure the _peak memory usage_ and _required time_ for both _inference_ and 
_training_.

<Tip>

Hereby, _inference_ is defined by a single forward pass, and _training_ is defined by a single forward pass and
backward pass.

</Tip>

===== Document 4 =====
layer is falsely activated during the forward pass, *i.e.* pass *self.training* to [PyTorch's functional 
dropout](https://pytorch.org/docs/stable/nn.functional.html?highlight=dropout#torch.nn.functional.dropout)

===== Document 5 =====
overhead. This is super helpful when you have activation checkpointing enabled, where we do a forward recompute and
backward passes a single layer granularity and want to keep the parameter in the forward recompute till the 
backward

===== Document 6 =====
This time we would illustrate the object `model_flax` without the weight matrices:

![alt text](https://raw.githubusercontent.com/patrickvonplaten/scientific_images/master/lm_flax_def.png)


Accordingly, the forward pass requires both `input_ids` as well as a dictionary consisting of the model's weights 
(called `state` here) to compute the `sequences`:

To get the initial `state` we need to explicitly do a forward pass by passing a dummy input:

===== Document 7 =====
to implement BigBird.
- The first goal should be to successfully run a forward pass using the RoBERTa checkpoint 
`bigbr_base/model.ckpt-0.data-00000-of-00001` and `bigbr_base/model.ckpt-0.index`.

===== Document 8 =====
For convolutions and linear layers there are 2x flops in the backward compared to the forward, which generally 
translates 
into ~2x slower (sometimes more, because sizes in the backward tend to be more awkward). Activations are usually 
bandwidth-limited, and it’s typical for an activation to have to read more data in the backward than in the forward
(e.g. activation forward reads once, writes once, activation backward reads twice, gradOutput and output of the 
forward,

===== Document 9 =====
We can either pass the name of a text dataset or a pretokenized dataset which speeds up training a bit.

Out: None

...

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
Based on the retrieved documentation, I can now provide a clear answer about which pass is slower during training. 
                                                                                                                   
Thought: The documentation, particularly Document 8, provides explicit information about the speed difference      
between forward and backward passes. It mentions that for linear layers and convolutions (which are fundamental    
components of transformer models):                                                                                 
1. Backward pass has 2x more floating point operations (FLOPs) than forward pass                                   
2. This generally translates to backward being approximately 2x slower                                             
3. For activations, the backward pass needs to read more data (reads twice - both gradOutput and output of the     
forward)                                                                                                           
                                                                                                                   
Let me provide the final answer.                                                                                   
                                                                                                                   
Code:                                                                                                              
```py                                                                                                              
final_answer("The backward pass is slower than the forward pass. Generally, it's approximately 2x slower due to    
having twice the number of floating point operations and requiring more memory reads for activation gradients.")   
```<end_code>                                                                                                      

╭─ Executing this code: ──────────────────────────────────────────────────────────────────────────────────────────╮
│   1 final_answer("The backward pass is slower than the forward pass. Generally, it's approximately 2x slower    │
│     due to having twice the number of floating point operations and requiring more memory reads for activation  │
│     gradients.")                                                                                                │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Out - Final answer: The backward pass is slower than the forward pass. Generally, it's approximately 2x slower due 
to having twice the number of floating point operations and requiring more memory reads for activation gradients.

[Step 1: Duration 4.53 seconds| Input tokens: 5,643 | Output tokens: 281]

The backward pass is slower than the forward pass. Generally, it's approximately 2x slower due to having twice the number of floating point operations and requiring more memory reads for activation gradients.
